# Dog Breeds - TF.image Augmentations Playground

This notebook allows you to experiment with different image augmentations.  It also shows how to use TF.py_function and a way to create a custom percentage wrapper.  It is NOT a step by step guide, but more of a playground.

The dataset illustrated is cats/dogs.  In real usage you would copy this notebook into your subdirectory and then change global parms as needed.  You will see a copy of this notebook in some of my folders.

To add/remove the augmentations, uncomment/comment the calls or add methods of your own.

The augmentation methods are the one in TF.image and other ones I have used in the past.  (A good list of what you can do to an image)

Here is the TF.image information

https://www.tensorflow.org/api_docs/python/tf/image

These are also good research links:

https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.shift.html

https://docs.scipy.org/doc/scipy-0.16.0/reference/generated/scipy.ndimage.interpolation.rotate.html

https://scikit-image.org/docs/dev/api/skimage.filters.html#skimage.filters.gaussian

https://www.tensorflow.org/api_docs/python/tf/py_function




### Processing for using Google Drive and normal includes

The notebook uses TensorFlow 2.x.  (Eager execution is enabled by default and we use the newer versions of tf.Data.)

I use Notebooks with Colab and on my local workstation, so I need to separate some logic to make it easier to run in both locations.

I was going to delete and just make Colab version, but that is not "real world."  You usually have multiple environments and I'm showing you how I accommodate different environments, you might need something different...

In [0]:
#"""
# Google Collab specific stuff....
from google.colab import drive
drive.mount('/content/drive')

import os
!ls "/content/drive/My Drive"

USING_COLLAB = True
# Force to use 2.x version of Tensorflow
%tensorflow_version 2.x
#"""

In [0]:
# Setup sys.path to find MachineLearning lib directory

# Check if "USING_COLLAB" is defined, if yes, then we are using Colab, otherwise set to False
try: USING_COLLAB
except NameError: USING_COLLAB = False

%load_ext autoreload
%autoreload 2

# set path env var
import sys
if "MachineLearning" in sys.path[0]:
    pass
else:
    print(sys.path)
    if USING_COLLAB:
        sys.path.insert(0, '/content/drive/My Drive/GitHub/MachineLearning/lib')  ###### CHANGE FOR SPECIFIC ENVIRONMENT
    else:
        sys.path.insert(0, '/Users/john/Documents/GitHub/MachineLearning/lib')  ###### CHANGE FOR SPECIFIC ENVIRONMENT
    
    print(sys.path)

In [0]:
# Normal includes...

from __future__ import absolute_import, division, print_function, unicode_literals

import os, sys, random, warnings, time, copy, csv
import numpy as np 

import IPython.display as display
from PIL import Image

import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
print(tf.__version__)

# This allows the runtime to decide how best to optimize CPU/GPU usage
AUTOTUNE = tf.data.experimental.AUTOTUNE

from TrainingUtils import *

#warnings.filterwarnings("ignore", category=DeprecationWarning)
#warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", "(Possibly )?corrupt EXIF data", UserWarning)

## General Setup

- Create a dictionary wrapped by a class for global values.  This is how I manage global vars in my notebooks.
- Load a couple of images that will be used to create a very simple dataset



In [0]:
# Set root directory path to data
if USING_COLLAB:
    ROOT_PATH = "/content/drive/My Drive/GitHub/MachineLearning/9-LibTest/Data"  ###### CHANGE FOR SPECIFIC ENVIRONMENT
else:
    ROOT_PATH = "/Users/john/Documents/GitHub/MachineLearning/9-LibTest/Data"  ###### CHANGE FOR SPECIFIC ENVIRONMENT
        
# Establish global dictionary
parms = GlobalParms(ROOT_PATH=ROOT_PATH,
                    TRAIN_DIR="CatDogLabeledVerySmall", 
                    NUM_CLASSES=120,
                    IMAGE_ROWS=224,
                    IMAGE_COLS=224,
                    IMAGE_CHANNELS=3,
                    BATCH_SIZE=32,
                    IMAGE_EXT=".jpg")

parms.print_contents()

In [0]:
# Simple helper method to display batches of images with labels....        
def show_batch(image_batch, label_batch, number_to_show=25, r=5, c=5, print_shape=False):
    show_number = min(number_to_show, parms.BATCH_SIZE)

    if show_number < 8: #if small number, then change row, col and figure size
        if parms.IMAGE_COLS > 64 or parms.IMAGE_ROWS > 64:
            plt.figure(figsize=(25,25)) 
        else:
            plt.figure(figsize=(10,10))  
        r = 4
        c = 2 
    else:
        plt.figure(figsize=(10,10))  

    if show_number == 1:
        image_batch = np.expand_dims(image_batch, axis=0)
        label_batch = np.expand_dims(label_batch, axis=0)

    for n in range(show_number):
        if print_shape:
            print("Image shape: {}  Max: {}  Min: {}".format(image_batch[n].shape, np.max(image_batch[n]), np.min(image_batch[n])))
        ax = plt.subplot(r,c,n+1)
        cmap="gray"
        if len(image_batch[n].shape) == 3:
            if image_batch[n].shape[2] == 3:
                cmap="viridis"
        plt.imshow(tf.keras.preprocessing.image.array_to_img(image_batch[n]), cmap=plt.get_cmap(cmap))
        plt.title(parms.CLASS_NAMES[np.argmax(label_batch[n])])
        plt.axis('off')

In [0]:
# Download dataset to local VM
import tensorflow_datasets as tfds
datasets, info = tfds.load(name='stanford_dogs', with_info=True, as_supervised=True)

In [0]:
# Set Class names...
parms.set_class_names(['Chihuahua', 'Japanese_spaniel', 'Maltese_dog', 'Pekinese', 'Shih-Tzu', 'Blenheim_spaniel', 'papillon', 'toy_terrier', 'Rhodesian_ridgeback', 'Afghan_hound', 'basset', 'beagle', 'bloodhound', 'bluetick', 'black-tan_coonhound', 'Walker_hound', 'English_foxhound', 'redbone', 'borzoi', 'Irish_wolfhound', 'Italian_greyhound', 'whippet', 'Ibizan_hound', 'Norwegian_elkhound', 'otterhound', 'Saluki', 'Scottish_deerhound', 'Weimaraner', 'Staffordshire_bullterrier', 'American_Staffordshire_terrier', 'Bedlington_terrier', 'Border_terrier', 'Kerry_blue_terrier', 'Irish_terrier', 'Norfolk_terrier', 'Norwich_terrier', 'Yorkshire_terrier', 'wire-haired_fox_terrier', 'Lakeland_terrier', 'Sealyham_terrier', 'Airedale', 'cairn', 'Australian_terrier', 'Dandie_Dinmont', 'Boston_bull', 'miniature_schnauzer', 'giant_schnauzer', 'standard_schnauzer', 'Scotch_terrier', 'Tibetan_terrier', 'silky_terrier', 'soft-coated_wheaten_terrier', 'West_Highland_white_terrier', 'Lhasa', 'flat-coated_retriever', 'curly-coated_retriever', 'golden_retriever', 'Labrador_retriever', 'Chesapeake_Bay_retriever', 'German_short-haired_pointer', 'vizsla', 'English_setter', 'Irish_setter', 'Gordon_setter', 'Brittany_spaniel', 'clumber', 'English_springer', 'Welsh_springer_spaniel', 'cocker_spaniel', 'Sussex_spaniel', 'Irish_water_spaniel', 'kuvasz', 'schipperke', 'groenendael', 'malinois', 'briard', 'kelpie', 'komondor', 'Old_English_sheepdog', 'Shetland_sheepdog', 'collie', 'Border_collie', 'Bouvier_des_Flandres', 'Rottweiler', 'German_shepherd', 'Doberman', 'miniature_pinscher', 'Greater_Swiss_Mountain_dog', 'Bernese_mountain_dog', 'Appenzeller', 'EntleBucher', 'boxer', 'bull_mastiff', 'Tibetan_mastiff', 'French_bulldog', 'Great_Dane', 'Saint_Bernard', 'Eskimo_dog', 'malamute', 'Siberian_husky', 'affenpinscher', 'basenji', 'pug', 'Leonberg', 'Newfoundland', 'Great_Pyrenees', 'Samoyed', 'Pomeranian', 'chow', 'keeshond', 'Brabancon_griffon', 'Pembroke', 'Cardigan', 'toy_poodle', 'miniature_poodle', 'standard_poodle', 'Mexican_hairless', 'dingo', 'dhole', 'African_hunting_dog'])

print("Classes: ", parms.NUM_CLASSES, 
      "   Labels: ", len(parms.CLASS_NAMES), 
      "  ", parms.CLASS_NAMES)


## Build an input pipeline

In [0]:

import random
import scipy.ndimage
from skimage.filters import gaussian

def image_blur(image):
    # Takes an image and applies Gaussian Blur using skimage filters.
    # Applies random +/- sigma_max to the image

    if bool(np.random.choice([0, 1], p=[0.7, 0.3])):  # change p values as needed . [0., 1.0] is always True
        sigma_max = 3.0
        sigma = random.uniform(0., sigma_max)  # change range or remove if want a fixed sigma value
        image = tf.image.convert_image_dtype(image, dtype=tf.int32)
        image = gaussian(image, sigma=sigma, multichannel=True)
        image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    return image

def image_aug_pre_cache(image: tf.Tensor) -> tf.Tensor:

    #######################################################
    # Blur using tf.py_function
    #######################################################
    im_shape = image.shape
    [image,] = tf.py_function(image_blur, [image], [tf.float32])  #parms must be tensors
    image.set_shape(im_shape)
    #######################################################

    image = tf.clip_by_value(image, 0., 1.)  # after majority of augmentations, clip back to 0, 1 before returning
    return image

def image_aug_post_cache(image: tf.Tensor) -> tf.Tensor:

    #######################################################
    # These are native tf.image methods
    #######################################################
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)

    #######################################################
    # random zoom - random crop + resize which will zoom the image
    #######################################################
    w = parms.IMAGE_COLS
    h = parms.IMAGE_ROWS
    p = 0.90
    image = tf.image.resize(tf.image.random_crop(image, (int(h*p), int(w*p), 3)), (h, w))
    #######################################################

    image = tf.clip_by_value(image, 0., 1.)  # after majority of augmentations, clip back to 0, 1 before returning
    return image

def image_normalize_0_1(image: tf.Tensor) -> tf.Tensor:
    image = tf.image.resize(image, (parms.IMAGE_ROWS, parms.IMAGE_COLS))
    image = tf.cast(image, tf.float32) / 255.
    return image

def image_normalize_0_1_to_1_neg_1(image: tf.Tensor) -> tf.Tensor:
    image = tf.subtract(image, 0.5)
    image = tf.multiply(image, 2.0)
    return image

def image_normalize_1_to_neg_1(image: tf.Tensor) -> tf.Tensor:
    image = tf.image.resize(image, (parms.IMAGE_ROWS, parms.IMAGE_COLS))
    image = tf.cast(image, tf.float32) / 255.
    image = tf.subtract(image, 0.5)
    image = tf.multiply(image, 2.0)
    return image

def process_train_pre_cache(image: tf.Tensor, label: tf.Tensor) -> tf.Tensor:
    image = image_normalize_0_1(image)
    image = image_aug_pre_cache(image)
    return image, label_to_onehot(label)

def process_train_post_cache(image: tf.Tensor, label: tf.Tensor) -> tf.Tensor:
    image = image_aug_post_cache(image)
    image = image_normalize_0_1_to_1_neg_1(image) # ImageNet needs 1 to -1
    return image, label

def process_val(image: tf.Tensor, label: tf.Tensor) -> tf.Tensor:
    image = image_normalize_1_to_neg_1(image)
    return image, label_to_onehot(label)

def label_to_onehot(label: tf.Tensor) -> tf.Tensor:
    return tf.one_hot(label, parms.NUM_CLASSES)


### Figure out - Train dataset and pre-cache mappings

Pipeline Flow:

create dataset -> map "process_train_pre_cache" -> repeat forever -> batch

This will illustrate resizing and pre-cache augmentations.


In [0]:
# Create Dataset from list of images
train_dataset = datasets['train']
train_len = info.splits['train'].num_examples

# Verify image paths were loaded and save one path for later in "some_image"
for f in train_dataset.take(2):
    some_image = f[0]
    print(f[1])

# map training images to processing, includes any augmentation
train_dataset = train_dataset.map(process_train_pre_cache, num_parallel_calls=AUTOTUNE)

# Verify the mapping worked
for image, label in train_dataset.take(1):
    print("Image shape: {}  Max: {}  Min: {}".format(image.numpy().shape, np.max(image.numpy()), np.min(image.numpy())))
    print("Label: ", np.argmax(label.numpy()), label.numpy())

# Repeat forever
train_dataset = train_dataset.repeat()

# set the batch size
train_dataset = train_dataset.batch(parms.BATCH_SIZE)

#create simple iterator - DELETE in training notebook
ds_iter_1 = iter(train_dataset)

In [0]:
# Show the images, execute this cell multiple times to see the images
 
image_batch, label_batch = next(ds_iter_1)
show_batch(image_batch.numpy(), label_batch.numpy(), number_to_show=4)

### Add final Train pre-cache and post-cache mappings

Pipeline Flow:

create dataset -> map "process_train_pre_cache" -> cache -> process_train_post_cache -> repeat forever -> batch

This will illustrate resizing and pre and post cache augmentations.

In [0]:
def cache_dataset(dataset, cache=False):
    if cache:
        if isinstance(cache, str):
            dataset = dataset.cache(cache)
        else:
            dataset = dataset.cache()
    return dataset

In [0]:
# Create Dataset from list of images
train_dataset = datasets['train']
train_len = info.splits['train'].num_examples

# Verify image paths were loaded and save one path for later in "some_image"
for f in train_dataset.take(2):
    some_image = f[0]
    print(f[1])

# map training images to pre-cache processing, includes any augmentation
train_dataset = train_dataset.map(process_train_pre_cache, num_parallel_calls=AUTOTUNE)

# Verify the mapping worked
for image, label in train_dataset.take(1):
    print("Image shape: {}  Max: {}  Min: {}".format(image.numpy().shape, np.max(image.numpy()), np.min(image.numpy())))
    print("Label: ", np.argmax(label.numpy()), label.numpy())

# cache
#train_dataset = cache_dataset(train_dataset, cache="./breedtrain2.tfcache")
train_dataset = cache_dataset(train_dataset) # no cache

# map training images to post-cache processing, includes any augmentation
train_dataset = train_dataset.map(process_train_post_cache, num_parallel_calls=AUTOTUNE)

# Repeat forever
train_dataset = train_dataset.repeat()

# set the batch size
train_dataset = train_dataset.batch(parms.BATCH_SIZE)

#create simple iterator - DELETE in training notebook
ds_iter_2 = iter(train_dataset)

In [0]:
# Show the images, execute this cell multiple times to see the images

image_batch, label_batch = next(ds_iter_2)
show_batch(image_batch.numpy(), label_batch.numpy(), print_shape=False)

### Create Val dataset mappings

Pipeline Flow:

create val dataset -> map "process_val" -> cache -> repeat forever -> batch

This will illustrate resizing for validation.

In [0]:
# Create Dataset from list of images
val_dataset = datasets['test']
val_len = info.splits['test'].num_examples

# Verify image paths were loaded and save one path for later in "some_image"
for f in val_dataset.take(2):
    some_image = f[0]
    print(f[1])

# map training images to processing, includes any augmentation
val_dataset = val_dataset.map(process_val, num_parallel_calls=AUTOTUNE)

# Verify the mapping worked
for image, label in val_dataset.take(1):
    print("Image shape: {}  Max: {}  Min: {}".format(image.numpy().shape, np.max(image.numpy()), np.min(image.numpy())))
    print("Label: ", np.argmax(label.numpy()), label.numpy())

# cache
#val_dataset = cache_dataset(val_dataset, cache="./breedval2.tfcache")
val_dataset = cache_dataset(val_dataset) # no cache

# Repeat forever
val_dataset = val_dataset.repeat()

# set the batch size
val_dataset = val_dataset.batch(parms.BATCH_SIZE)

#create simple iterator - DELETE in training notebook
ds_iter_3 = iter(val_dataset)
ds_iter_3 = iter(val_dataset)

In [0]:
# Show the images, execute this cell multiple times to see the images

image_batch, label_batch = next(ds_iter_3)
show_batch(image_batch.numpy(), label_batch.numpy())

### Final Thoughts.....

Needed to split augmentation into a pre/post methods because the Blur takes long, so wanted to cache resizing and Blur, but random apply the other augmentations along with the final change to have values between 1 and -1.
